In [3]:
import requests
import csv
from bs4 import BeautifulSoup

In [4]:
url_base = "https://www.euromillones.com.es/historico/resultados-euromillones-{}.html"

In [10]:
def url_ano(url_base, ano):
    # Entrar com a url base que necessita apenas alterar o ano
    return url_base.format(ano)

In [41]:
def status_site(url_base, ano):
    # Retorna o status em caso de erro
    url = url_ano(url_base, ano)
    response = requests.get(url)
    status = response.status_code
    
    if status != 200:
        print(f"Erro ao baixar {ano}: {status}")
        return url, response
    else:
        return url, response

In [42]:
def status_tabela(url_base, ano):
    
    url, response = status_site(url_base, ano)
    soup = BeautifulSoup(response.text, 'html.parser')
    tabela = soup.find('table', class_='histoeuro')
    
    if not tabela:
        print(f"Tabela não encontrada para o ano {ano}")
    else:
        return tabela.find_all('tr')[2:] # Ignora os cabeçalhos


In [79]:
def sorteio_padrao(linhas, i):
    
    """Remove a primeira coluna <td class='nmt' rowspan='2'> se presente."""
    
    tds = linhas[i].find_all("td")
    
    if "nmt" in tds[0].get("class", []):
        tds.pop(0)  # Remove a primeira célula se for um identificador de semana
    return tds

In [91]:
def limpar_linhas(concurso, data, numeros, estrelas):
    
    """Remove linhas que contenham valores vazios."""
    
    if not concurso or not data or any(not num for num in numeros + estrelas):
        return None
    return [concurso, data] + numeros + estrelas

In [108]:
def sorteios_anuais(url_base, ano):
    
    linhas = status_tabela(url_base, ano)
    dados = []

    for i in range(0, len(linhas)):
        try:
            sorteio = sorteio_padrao(linhas, i)
            concurso = sorteio[0].text.strip()
            data = sorteio[1].text.strip()
            numeros = [sorteio[j].text.strip() for j in range(2, 7)]
            estrelas = [sorteio[7].text.strip(), sorteio[8].text.strip()]

            nova_linha = limpar_linhas(concurso, data, numeros, estrelas)

            if nova_linha:
                dados.append(nova_linha)

        except Exception as e:
            print(f"Erro ao processar linha {e} no ano {ano}")
            continue
    
    return dados

In [109]:
def salvar_csv(dados, ano):
    nome_arquivo = f"{ano}_eurom.csv"
    
    with open(nome_arquivo, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(["Concurso", "Data", "Num1", "Num2", "Num3", "Num4", "Num5", "Estrela1", "Estrela2"])
        writer.writerows(dados)
    
    print(f"Salvo: {nome_arquivo}")

In [110]:
for ano in range(2004, 2026):
    dados = sorteios_anuais(url_base, ano)
    salvar_csv(dados, ano)

Salvo: 2004_eurom.csv
Salvo: 2005_eurom.csv
Salvo: 2006_eurom.csv
Salvo: 2007_eurom.csv
Salvo: 2008_eurom.csv
Erro ao processar linha list index out of range no ano 2009
Salvo: 2009_eurom.csv
Erro ao processar linha list index out of range no ano 2010
Erro ao processar linha list index out of range no ano 2010
Salvo: 2010_eurom.csv
Salvo: 2011_eurom.csv
Salvo: 2012_eurom.csv
Salvo: 2013_eurom.csv
Salvo: 2014_eurom.csv
Salvo: 2015_eurom.csv
Salvo: 2016_eurom.csv
Salvo: 2017_eurom.csv
Erro ao processar linha list index out of range no ano 2018
Erro ao processar linha list index out of range no ano 2018
Salvo: 2018_eurom.csv
Salvo: 2019_eurom.csv
Salvo: 2020_eurom.csv
Salvo: 2021_eurom.csv
Salvo: 2022_eurom.csv
Salvo: 2023_eurom.csv
Salvo: 2024_eurom.csv
Salvo: 2025_eurom.csv
